<a href="https://colab.research.google.com/github/egarmir/TFM_LastMile_Pulse/blob/main/LastMile%20Pulse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio

In [ ]:
import gradio as gr
import pandas as pd
import joblib
import os
import plotly.express as px
import folium
from folium.plugins import HeatMap

# CARGA DE MODELOS
ruta_drive = '/content/drive/MyDrive/GRADIO - LastMilePulse/'
clf_model = joblib.load(os.path.join(ruta_drive, 'pipeline_clasificacion_lade.joblib'))
reg_model = joblib.load(os.path.join(ruta_drive, 'pipeline_regresion_lade.joblib'))

# FUNCIONES

# Función Predicción Individual
def predecir_manual(hour, distance, orders, aoi, day_name, mean_delay, volume):
    try:
        dias_dict = {"Lunes": 0, "Martes": 1, "Miércoles": 2, "Jueves": 3, "Viernes": 4, "Sábado": 5, "Domingo": 6}
        day_week_num = dias_dict[day_name]

        datos = {
            'hour': hour, 'distance_km': distance, 'orders_by_courier': orders,
            'aoi_type': aoi, 'day_of_week': day_week_num,
            'courier_mean_delay': mean_delay, 'courier_volume': volume,
            'courier_sla_rate': 0.85,
            'aoi_congestion': 0.5 if (13 <= hour <= 21) else 0.2,
            'is_weekend': 1 if day_week_num >= 5 else 0,
            'is_peak_hour': 1 if (13 <= hour <= 15) or (20 <= hour <= 22) else 0
        }

        df = pd.DataFrame([datos])
        cols_modelo = ['aoi_type', 'courier_sla_rate', 'is_weekend', 'orders_by_courier',
                       'day_of_week', 'aoi_congestion', 'is_peak_hour', 'courier_mean_delay',
                       'courier_volume', 'distance_km', 'hour']

        prob = clf_model.predict_proba(df[cols_modelo])[0, 1]
        minutos = reg_model.predict(df[cols_modelo])[0]

        umbral_minutos = 90

        if prob >= 0.34 or minutos >= umbral_minutos:
            estado = "🔴 RIESGO DE RETRASO"
        else:
            estado = "🟢 SIN RIESGO"

        # 3. Formateo de salida
        prob_str = f"{prob:.1%}"
        minutos_str = f"{round(minutos, 1)} min"

        return estado, prob_str, minutos_str

    except Exception as e:
        return f"Error: {e}", "---", "---"

# B) Función Análisis General (CSV)
def analizar_csv_visual(archivo, hora_filtro):
    try:
        df = pd.read_csv(archivo.name)

        # 1. Preparar datos para predicción masiva
        cols_modelo = ['aoi_type', 'courier_sla_rate', 'is_weekend', 'orders_by_courier',
                       'day_of_week', 'aoi_congestion', 'is_peak_hour', 'courier_mean_delay',
                       'courier_volume', 'distance_km', 'hour']

        # Si faltan columnas calculadas, las creamos rápido
        if 'is_weekend' not in df.columns: df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
        if 'is_peak_hour' not in df.columns: df['is_peak_hour'] = df['hour'].apply(lambda x: 1 if (13 <= x <= 15) or (20 <= x <= 22) else 0)

        df['minutos_pred'] = reg_model.predict(df[cols_modelo])

        # 2. Crear HISTOGRAMA (Distribución de retrasos por AOI)
        fig_hist = px.histogram(df, x="minutos_pred", color="aoi_type",
                               title="Distribución de Retrasos Estimados",
                               labels={'minutos_pred': 'Minutos de Retraso'},
                               barmode='overlay', opacity=0.7)

        # Crear MAPA DE CALOR (Heatmap)
        # Filtramos por la hora seleccionada para ver "zonas calientes"
        df_mapa = df[df['hour'] == hora_filtro]

        # Coordenadas aproximadas para centrar el mapa (Chongqing/Shanghai)
        # Ajustamos a Shanghai por defecto si no detectamos la ciudad
        lat_center, lon_center = df_mapa['lat'].mean(), df_mapa['lng'].mean()
        m = folium.Map(location=[lat_center, lon_center], zoom_start=11, tiles='cartodbpositron')

        # Crear capa de calor: [lat, lon, peso]
        heat_data = [[row['lat'], row['lng'], row['minutos_pred']] for index, row in df_mapa.iterrows()]
        HeatMap(heat_data, radius=15, blur=10).add_to(m)

        # Guardar mapa como HTML para Gradio
        mapa_html = m._repr_html_()

        return fig_hist, mapa_html

    except Exception as e:
        print(f"Error: {e}")
        return None, f"<h3>Error al procesar: Asegúrate de que el CSV tenga columnas 'lat' y 'lng'</h3>"

# --- 3. INTERFAZ ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🚚 LaDe Analytics: Last Mile Pulse")

    with gr.Tab("🎯 Análisis Individual"):
        with gr.Row():
            with gr.Column():
                h_in = gr.Number(label="Hora", value=14)
                d_in = gr.Number(label="Distancia (km)", value=5.0)
                aoi_in = gr.Dropdown(choices=[1, 2, 3, 4, 14], value=1, label="AOI")
                dw_in = gr.Dropdown(choices=["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"], value="Lunes", label="Día")
            with gr.Column():
                o_in = gr.Number(label="Pedidos asignados", value=10)
                md_in = gr.Number(label="Retraso Hist.", value=15)
                vol_in = gr.Number(label="Volumen Total", value=100)

        btn_ind = gr.Button("⚡ CALCULAR PREDICCIÓN", variant="primary")

        with gr.Row():
            out_e = gr.Textbox(label="Evaluación")
            out_p = gr.Textbox(label="Probabilidad")
            out_m = gr.Textbox(label="Estimación")

        btn_ind.click(predecir_manual, [h_in, d_in, o_in, aoi_in, dw_in, md_in, vol_in], [out_e, out_p, out_m])

    with gr.Tab("📊 Dashboard General"):
        gr.Markdown("### Análisis de Zonas Críticas (Heatmap)")
        with gr.Row():
            file_input = gr.File(label="Sube tu CSV (debe incluir lat/lng)")
            hour_map = gr.Number(0, 23, value=14, label="Filtrar Mapa por Hora")

        btn_gen = gr.Button("Generar Análisis Visual")

        with gr.Row():
            plot_hist = gr.Plot(label="Histograma de Retrasos")

        gr.Markdown("#### Mapa de Calor de Retrasos")
        plot_map = gr.HTML()

        btn_gen.click(analizar_csv_visual, inputs=[file_input, hour_map], outputs=[plot_hist, plot_map])

demo.launch(debug=True, share=True)

/tmp/ipython-input-2782898007.py:101: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c016856e1c7b23433b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/util